# 计算项目活跃度

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import ast 
import statistics
from datetime import datetime 

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def activity(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,prefix):

    df=pd.read_csv('activity.csv')
    df_extra=pd.read_csv('death.csv')
    df_size=pd.read_csv('size.csv')
    df=pd.DataFrame(df)
    df_extra=pd.DataFrame(df_extra)
    df_size=pd.DataFrame(df_size)
    #采用contributor busfactor inact new
    #bus不可失去即核心
    # 防止没有搜寻到路径，则将该方面评分设置为0
    data_con = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_bus = pd.read_csv(input_file_path2)['指标值'] if input_file_path2 else np.zeros(300, dtype=int)
    data_new = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_inact = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_part = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)

    part_max=data_part.max()

    if part_max < 10 :
        size =1
    elif part_max < 100 :
        size =2
    elif part_max <1000 :
        size =3
    elif part_max <10000 :
        size =4
    else :
        size =5

    row_exists=(df_size['指标名称']==prefix).any()

    if row_exists:
        df_size.loc[df_size['指标名称'] == prefix, '指标值']=size
    # 保存更改回 CSV 文件
    df_size.to_csv('size.csv', index=False)

    death_time=0
    data_part_live=[]
    data_bus_live=[]
    data_con_live=[]
    data_inact_live=[]
    data_new_live=[]
    for i in range(len(data_part)) :
        if (data_part[i]/part_max) < 0.1:
            death_time+=1
        else:
            data_part_live.append(data_part[i])
            data_bus_live.append(data_bus[i])
            data_con_live.append(data_con[i])
            data_inact_live.append(data_inact[i])
            data_new_live.append(data_new[i])
    # 计算 extra
    death=death_time/len(data_part)
    row_exists=(df_extra['指标名称']==prefix).any()

    if row_exists:
        df_extra.loc[df_extra['指标名称'] == prefix, '指标值']=death*10
    # 保存更改回 CSV 文件
    df_extra.to_csv('death.csv', index=False)


    s_con=[]
    s_bus=[]
    s_new=[]
    s_inact=[]

    for i in range(len(data_con_live)): 
        if data_bus_live[i]!=0:
            s_con.append(data_con_live[i]/data_bus_live[i])        # 贡献者 占总人数的比例
        else:
            s_con.append(0)
        if data_part_live[i]!=0:
            s_bus.append(data_bus_live[i]/data_part_live[i])       # bus_factor 占总参与人数的比例
            s_new.append(data_new_live[i]/data_part_live[i])       # 新贡献者 占总人数的比例
            s_inact.append(data_inact_live[i]/data_part_live[i])   # 不活跃的贡献者 占总人数的比例
        else:
            s.bus.append(0)
            s_new.append(0)
            s_inact.append(0)

    avg1=np.mean(s_bus)
    avg2=np.mean(s_inact)
    avg3=np.mean(s_new)
    avg4=np.mean(s_con)
    avg5=1-avg2
    avg6=1-avg1
    if avg5<0:
        avg5=0
    if avg6<0:
        avg6=0
    final=avg6/6+avg3/6+avg5/6+avg4/2
    #print(final)
    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final*10

    # 保存更改回 CSV 文件
    df.to_csv('activity.csv', index=False)

def time(input_file_path10,input_file_path5,prefix):

    df_time=pd.read_csv('time.csv')
    df_time=pd.DataFrame(df_time)

    data_active=pd.read_csv(input_file_path10)['指标值']
    data_part = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)

    death_time=0
    data_part_live=[]
    data_active_live=[]
    part_max=data_part.max()
    for i in range(len(data_part)):
        if (data_part[i]/part_max) < 0.1:
            death_time+=1
        else:
            data_part_live.append(data_part[i])
            data_active_live.append(data_active[i])

    s_sum=[]
    s_grade=[]

    for i in range(len(data_active_live)):
        s_data=ast.literal_eval(data_active_live[i])
        s_sum.append(np.sum(s_data))
        s_grade.append(10*s_sum[i]/600)

    avg=np.mean(s_grade)
    row_exists=(df_time['指标名称']==prefix).any()

    if row_exists:
        df_time.loc[df_time['指标名称'] == prefix, '指标值']=avg

    # 保存更改回 CSV 文件
    df_time.to_csv('time.csv', index=False)

def request(input_file_path6,input_file_path7,input_file_path8,input_file_path9,prefix):

    df=pd.read_csv('request.csv')
    df=pd.DataFrame(df)

    df_dura = pd.read_csv(input_file_path6) if input_file_path6 else np.zeros(300, dtype=int)
    df_respontime =pd.read_csv(input_file_path7) if input_file_path7 else np.zeros(300, dtype=int)
    df_request=pd.read_csv(input_file_path8)['指标值'] if input_file_path8 else np.zeros(300, dtype=int)
    df_accepted=pd.read_csv(input_file_path9)['指标值'] if input_file_path9 else np.zeros(300, dtype=int)

    data_request=[]
    data_accepted=[]
    for i in range(len(df_request)):
        data_request.append(df_request[i].item())
    for i in range(len(df_accepted)):
        data_accepted.append(df_accepted[i].item())

    if(input_file_path6):
        data_dura=ast.literal_eval(df_dura['指标值'][0])
        data_respontime=ast.literal_eval(df_respontime['指标值'][0])

        times_floats=[]
        for key, value in data_dura.items():
            if isinstance(key, str) and len(key) == 7 and key[4] == '-':  # 简单检查是否为YYYY-MM格式
                try:
                    # 将字符串时间转换为datetime对象
                    date_obj = datetime.strptime(key, '%Y-%m')
                    # 尝试将值转换为float
                    float_value = float(value)
                    times_floats.append((date_obj, float_value))
                except (ValueError, TypeError):
                    # 如果转换失败，跳过该项
                    continue
        # 按照日期排序
        times_floats.sort(key=lambda x: x[0])
        # 分离时间和值到两个独立的列表中
        dates_dura, values_dura = zip(*times_floats) if times_floats else ([], [])
        
        # 过滤并转换时间和值
        times_floats = []

        for key, value in data_respontime.items():
            if isinstance(key, str) and len(key) == 7 and key[4] == '-':  # 简单检查是否为YYYY-MM格式
                try:
                    # 将字符串时间转换为datetime对象
                    date_obj = datetime.strptime(key, '%Y-%m')
                    # 尝试将值转换为float
                    float_value = float(value)
                    times_floats.append((date_obj, float_value))
                except (ValueError, TypeError):
                    # 如果转换失败，跳过该项
                    continue

        # 按照日期排序
        times_floats.sort(key=lambda x: x[0])
        # 分离时间和值到两个独立的列表中
        dates_respontime, values_respontime = zip(*times_floats) if times_floats else ([], [])

        s_value_dura=[]
        s_value_respontime=[]
        for i in range(len(values_dura)):
            if values_dura[i]>10 :
                s_value_dura.append(10)
            else:
                s_value_dura.append(values_dura[i])
        for i in range(len(values_respontime)):
            if values_respontime[i]>20 :
                s_value_respontime.append(20)
            else:
                s_value_respontime.append(values_respontime[i])
        avg2=np.mean(s_value_respontime)
        avg3=np.mean(s_value_dura)    
    else:
        avg2=20
        avg3=10


    s_request=[]
    for i in range(len(data_request)):
        if data_request[i]!=0 :
            s_request.append(data_accepted[i]/data_request[i])
        else :
            s_request.append(0)
    avg1=np.mean(s_request)
    avg4=20-avg2
    avg5=10-avg3
    if avg4 < 0:
        avg4=0
    if avg5 < 0 :
        avg5=0
    
    final=(avg1*10+avg4/2+avg5)/3
    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final

    # 保存更改回 CSV 文件
    df.to_csv('request.csv', index=False)


def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'contributor.csv', 'bus_factor.csv', 'new_contributors.csv',
        'inactive_contributors.csv', 'participants.csv',
        'change_request_resolution_duration.csv',
        'change_request_response_time.csv', 'change_requests.csv',
        'change_requests_accepted.csv','active_dates_and_times.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('contributor.csv')
        input_file_path2=paths.get('bus_factor.csv')
        input_file_path3=paths.get('new_contributors.csv')
        input_file_path4=paths.get('inactive_contributors.csv')
        input_file_path5=paths.get('participants.csv')
        input_file_path6=paths.get('change_request_resolution_duration.csv')
        input_file_path7=paths.get('change_request_response_time.csv')
        input_file_path8=paths.get('change_requests.csv')            
        input_file_path9=paths.get('change_requests_accepted.csv')
        input_file_path10=paths.get('active_dates_and_times.csv')
        activity(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,prefix)
        time(input_file_path10,input_file_path5,prefix)
        request(input_file_path6,input_file_path7,input_file_path8,input_file_path9,prefix)

# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)

# 计算社区参与度

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import ast 
import statistics
from datetime import datetime 

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def issue(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,prefix):

    df=pd.read_csv('issue.csv')
    df=pd.DataFrame(df)

    data_comment = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_dura = pd.read_csv(input_file_path2)['指标值']  if input_file_path2 else np.zeros(300, dtype=int)
    data_respontime = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_new = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_closed = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)
    
    if input_file_path2:

        data_duration=ast.literal_eval(data_dura[0])

        # 过滤并转换时间和值
        times_floats = []
        for key, value in data_duration.items():
            if isinstance(key, str) and len(key) == 7 and key[4] == '-':  # 简单检查是否为YYYY-MM格式
                try:
                    # 将字符串时间转换为datetime对象
                    date_obj = datetime.strptime(key, '%Y-%m')
                    # 尝试将值转换为float
                    float_value = float(value)
                    times_floats.append((date_obj, float_value))
                except (ValueError, TypeError):
                    # 如果转换失败，跳过该项
                    continue
        # 按照日期排序
        times_floats.sort(key=lambda x: x[0])
        # 分离时间和值到两个独立的列表中
        dates_duration, values_duration = zip(*times_floats) if times_floats else ([], [])
       
        s_duration=[]
        
        for i in range(len(values_duration)):
            if values_duration[i]>10 :
                s_duration.append(10)
            else:
                s_duration.append(values_duration[i])
        avg4=np.mean(s_duration)
    else :
        avg4=0    
       
    if input_file_path3:
        data_response=ast.literal_eval(data_respontime[0]) 
        times_floats = []
        for key, value in data_response.items():
            if isinstance(key, str) and len(key) == 7 and key[4] == '-':  # 简单检查是否为YYYY-MM格式
                try:
                    # 将字符串时间转换为datetime对象
                    date_obj = datetime.strptime(key, '%Y-%m')
                    # 尝试将值转换为float
                    float_value = float(value)
                    times_floats.append((date_obj, float_value))
                except (ValueError, TypeError):
                    # 如果转换失败，跳过该项
                    continue
        # 按照日期排序
        times_floats.sort(key=lambda x: x[0])
        # 分离时间和值到两个独立的列表中
        dates_response, values_response = zip(*times_floats) if times_floats else ([], [])
        s_response=[]
        for i in range(len(values_response)):
                if values_response[i]>20 :
                    s_response.append(20)
                else:
                    s_response.append(values_response[i]) 
        avg3=np.mean(s_response)
        avg3=avg3/2
    else:
        avg3=0

    s_closed=[]
    s_new=[]
   
    for i in range(len(data_comment)):
        if data_comment[i]!=0:
            s_closed.append(data_closed[i]/data_comment[i])
            s_new.append(data_new[i]/data_comment[i])
        else:
            s_closed.append(0)
            s_new.append(0)

    avg1=np.mean(s_closed)
    avg2=np.mean(s_new)

    final=avg1*10/6+avg2*10/3+avg3/4+avg4/4

    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final

    # 保存更改回 CSV 文件
    df.to_csv('issue.csv', index=False)

def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'issue_comments.csv','issue_resolution_duration.csv','issue_response_time.csv',
        'issues_new.csv','issues_closed.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('issue_comments.csv')
        input_file_path2=paths.get('issue_resolution_duration.csv')
        input_file_path3=paths.get('issue_response_time.csv')
        input_file_path4=paths.get('issues_new.csv')
        input_file_path5=paths.get('issues_closed.csv')
        issue(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,prefix)

# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)

# 计算代码质量

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import ast 
import statistics
from datetime import datetime 

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def quality(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,input_file_path6,prefix):

    df=pd.read_csv('quality.csv')
    df=pd.DataFrame(df)

    data_add = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_remove = pd.read_csv(input_file_path2)['指标值'] if input_file_path2 else np.zeros(300, dtype=int)
    data_sum = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_review = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_request = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)
    data_accepted = pd.read_csv(input_file_path6)['指标值'] if input_file_path6 else np.zeros(300, dtype=int)


    sum_request=sum(data_request)
    sum_review=sum(data_review)
    if(sum_request!=0):
        avg1=sum_review/sum_request
    else :
        avg1=0
    s_request=[]
    for i in range(len(data_request)) :
        if data_request[i]!=0:
            s_request.append(data_accepted[i]/data_request[i])
        else:
            s_request.append(0)
    avg2=np.mean(s_request)

    s_add=[]
    s_remove=[]
    for i in range(len(data_sum)):
        if data_sum[i]!=0:
            s_add.append(data_add[i]/np.abs(data_sum[i]))
            s_remove.append(data_remove[i]/np.abs(data_sum[i]))
        else :
            s_add.append(0)
            s_remove.append(0)
    avg3=np.mean(s_add)
    avg4=np.mean(s_remove)
    avg5=avg3-avg4
    final=avg1/4+avg2/4+abs(avg5/2)
    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final*10

    # 保存更改回 CSV 文件
    df.to_csv('quality.csv', index=False)

def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'code_change_lines_add.csv','code_change_lines_remove.csv','code_change_lines_sum.csv',
        'change_requests_reviews.csv','change_requests.csv','change_requests_accepted.csv',
        'active_dates_and_times.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('code_change_lines_add.csv')
        input_file_path2=paths.get('code_change_lines_remove.csv')
        input_file_path3=paths.get('code_change_lines_sum.csv')
        input_file_path4=paths.get('change_requests_reviews.csv')
        input_file_path5=paths.get('change_requests.csv')
        input_file_path6=paths.get('change_requests_accepted.csv')

        quality(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,input_file_path6,prefix)

# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)

# 计算用户反馈

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import ast 
import statistics
from datetime import datetime 

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def feedback(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,input_file_path6,input_file_path7,prefix):

    df=pd.read_csv('feedback.csv')
    df=pd.DataFrame(df)

    data_part = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_attention = pd.read_csv(input_file_path2)['指标值'] if input_file_path2 else np.zeros(300, dtype=int)
    data_new = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_comment = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_stars = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)
    data_fork = pd.read_csv(input_file_path6)['指标值'] if input_file_path6 else np.zeros(300, dtype=int)
    data_closed = pd.read_csv(input_file_path7)['指标值'] if input_file_path7 else np.zeros(300, dtype=int)
    
    s_closed=[]
    s_new=[]
    s_attention=[]
    s_stars=[]
    s_fork=[]
    for i in range(len(data_comment)) :
        if data_comment[i]!=0:
            s_closed.append(data_closed[i]/data_comment[i])
            s_new.append(data_new[i]/data_comment[i])
        else:
            s_closed.append(0)
            s_new.append(0)

    avg1=np.mean(s_closed)
    avg2=np.mean(s_new)

    for i in range(len(data_part)):
        if data_part[i]!=0:
            s_attention.append(data_attention[i]/data_part[i])
        else:
            s_attention.append(0)
    
    for i in range(len(data_attention)):
        if data_attention[i]!=0:
            s_stars.append(data_stars[i]/data_attention[i])
            s_fork.append(data_fork[i]/data_attention[i])
        else:
            s_stars.append(0)
            s_fork.append(0)

    avg3=np.mean(s_attention)
    avg4=np.mean(s_stars)
    avg5=np.mean(s_fork)
    final=avg1*1/9+avg2*2/9+avg3*1/3+avg4*1/6+avg5*1/6


    # 获取仓库名
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final*10

    # 保存更改回 CSV 文件
    df.to_csv('feedback.csv', index=False)

def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'participants.csv','attention.csv','issues_new.csv',
        'issue_comments.csv','stars.csv','technical_fork.csv','issues_closed.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('participants.csv')
        input_file_path2=paths.get('attention.csv')
        input_file_path3=paths.get('issues_new.csv')
        input_file_path4=paths.get('issue_comments.csv')
        input_file_path5=paths.get('stars.csv')
        input_file_path6=paths.get('technical_fork.csv')
        input_file_path7=paths.get('issues_closed.csv')
        feedback(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5,input_file_path6,input_file_path7,prefix)

# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)